In [ ]:
'''
參考連結:
[1] Implement Okapi BM25 variants in Gensim #3304
https://github.com/piskvorky/gensim/pull/3304
[2] 法規名稱： 銀行年報應行記載事項準則 (修正日期：	民國 113 年 02 月 19 日)
https://law.moj.gov.tw/LawClass/LawAll.aspx?PCODE=G0380104
[3] 法規名稱： 公開發行公司年報應行記載事項準則 (修正日期：	民國 112 年 11 月 10 日)
https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=G0400022
[4] gensim - models.tfidfmodel
https://radimrehurek.com/gensim/models/tfidfmodel.html
'''

In [1]:
import re
import sqlite3
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import jieba
import heapq

In [2]:
# 資料庫連線
db_file = '2633.db'
conn = sqlite3.connect(db_file)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [ ]:
# 分詞設定
jieba.load_userdict("userdict.txt")

In [4]:
# 匯出所有資料
sql = f'''
SELECT *
FROM `pdf`
ORDER BY `id` ASC
'''
stmt = cursor.execute(sql)
list_data = []
list_page_num = []
for row in stmt.fetchall():
    text_content = row['text_content']
    table_format = row['table_format']
    # if table_format == None: 
    #     list_data.append(text_content)
    # else:
    #     list_data.append(text_content +  table_format)
    list_data.append(text_content)
    list_page_num.append(row['page_num'])

In [5]:
# 關閉 sqlite
cursor.close()
conn.close()

In [6]:
# 斷詞
list_seq = []
for seq in list_data:
    seq = re.sub(r"\s| |\n", "", seq) # 簡單清理
    list_seq.append(list(jieba.cut(seq, HMM=False)))

In [7]:
# 用 BM25 模型計算相似度(相關性)

'''
定義函式 bm25_similarity：
- list_seq 是一個文件的列表，每個文件都是一個詞的列表；
- query 是查詢字符串。
'''
def bm25_similarity(list_seq, query):
    '''
    建立詞典：
    - 使用 list_seq 中的所有詞彙建立一個 Dictionary 物件，為每個詞彙分配一個唯一的ID。
    '''
    dictionary = Dictionary(list_seq)

    '''
    初始化 BM25 模型：
    - 使用先前建立的詞典，來初始化一個 OkapiBM25Model 物件，這是用於資訊檢索 (Information Retrieval) 的統計模型。
    '''
    bm25_model = OkapiBM25Model(dictionary=dictionary)

    '''
    建立 BM25 語料庫：
    - dictionary.doc2bow：將每個文件（詞的列表）轉換為詞袋（Bag-of-Words）表示，即每個詞的ID及其在文件中的出現次數。
    - map 函式：對 list_seq 中的每個文件應用 doc2bow 轉換。
    - bm25_model[...]：對轉換後的語料庫應用 BM25 權重，得到加權的語料庫 bm25_corpus。
    '''
    bm25_corpus = bm25_model[list(map(dictionary.doc2bow, list_seq))]

    '''
    SparseMatrixSimilarity：創建一個稀疏矩陣相似度對象，用於快速計算查詢與文件之間的相似度。
    - bm25_corpus：BM25 加權的語料庫。
    - num_docs：文件數量。
    - num_terms：詞典中詞的數量。
    - normalize_queries=False：不對查詢進行 normalization。
    - normalize_documents=False：不對文件進行 normalization。
    '''
    bm25_index = SparseMatrixSimilarity(
        bm25_corpus, 
        num_docs=len(list_seq), 
        num_terms=len(dictionary), 
        normalize_queries=False, 
        normalize_documents=False
    )

    '''
    對查詢進行分詞：
    - 使用 jieba.lcut 對查詢字符串進行分詞。
    '''
    query = jieba.lcut(query)

    '''
    建立 TF-IDF 模型：
    - 參數 smartirs='bnn'：指定權重計算的方式，其中 'bnn' 表示二值化的詞頻和未正則化的逆向文件頻率 (IDF)。
    '''
    tfidf_model = TfidfModel(
        dictionary=dictionary, 
        smartirs='bnn' # default: nfc
    )

    '''
    轉換查詢向量：
    - dictionary.doc2bow(query)：將查詢詞列表轉換為詞袋表示。
    - tfidf_model[...]：對詞袋表示的查詢應用 TF-IDF 權重，得到加權的查詢向量 tfidf_query。
    '''
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]

    '''
    計算相似度：
    - 使用之前建立的 bm25_index，計算查詢向量與語料庫中每個文件的相似度。
    '''
    similarities = bm25_index[tfidf_query]
    
    return similarities

In [21]:
# 顯示前 top_k 高相似度的文章
def get_page_score(similarities, top_k):
    '''
    (文章索引, 查詢後的相似分數)
    [(0, 0.0), (1, 0.0), (2, 0.0), (3, 3.4350295), (4, 2.4153714), (5, 0.0), (6, 0.0), ...]
    '''
    similarities = list(enumerate(similarities))

    '''
    前 top_k 高相似度的文章
    [(176, 15.524609), (177, 9.0146), (116, 7.6217546), (100, 6.096363), (159, 4.855205)]
    '''
    top_items  = heapq.nlargest(
        top_k, 
        similarities,
        key=lambda x: x[1]
    )

    return [(doc_index + 1, score) for doc_index, score in top_items]

In [22]:
# 基本設定
top_k = 5

In [ ]:
'''
第一類
18.4	(四)營運概況 4.環保支出資訊：最近年度及截至年報刊印日止，因污染環境所遭受之損失（包括賠償及環境保護稽查結果違反環保法規事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
18.5.1	5.勞資關係 (1)列示公司各項員工福利措施、進修、訓練、退休制度與其實施情形，以及勞資間之協議與各項員工權益維護措施情形。
18.5.2	(2)列明最近年度及截至年報刊印日止，因勞資糾紛所遭受之損失（包括勞工檢查結果違反勞動基準法事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。

第二類
11.1	3.公司資本及股份 (1)股本來源是否按附表五揭露。
11.2	(2)股東結構是否按附表六揭露。
11.3	(3)普通股及特別股之股權分散情形是否按附表七揭露。
11.4	(4)主要股東名單是否按附表八揭露，應列明股權比例達百分之五以上之股東，如不足十名，應揭露至股權比例占前十名之股東名稱、持股數額及比例。
11.5	(5)最近二年度每股資料是否按附表九揭露。
11.9.1	(9)公司買回本公司股份情形 A.已執行完畢者：是否按附表十(1)揭露。
11.9.2	B.尚在執行中者：是否按附表十(2)揭露。

第四類
18.1.1	(四)營運概況 1.業務內容 (1)業務範圍。
18.1.2	(2)產業概況。
18.1.3	(3)技術及研發概況。
18.1.4	(4)長、短期業務發展計畫。
'''

In [ ]:
# 第 1 類: 18.4
query = '''
(四)營運概況 4.環保支出資訊：最近年度及截至年報刊印日止，因污染環境所遭受之損失（包括賠償及環境保護稽查結果違反環保法規事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
'''
query = '''
營運概況 環保支出資訊 污染環境
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 1 類: 18.5.1
query = '''
5.勞資關係 (1)列示公司各項員工福利措施、進修、訓練、退休制度與其實施情形，以及勞資間之協議與各項員工權益維護措施情形。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 1 類: 18.5.2
query = '''
(2)列明最近年度及截至年報刊印日止，因勞資糾紛所遭受之損失（包括勞工檢查結果違反勞動基準法事項，應列明處分日期、處分字號、違反法規條文、違反法規內容、處分內容），並揭露目前及未來可能發生之估計金額與因應措施，如無法合理估計者，應說明其無法合理估計之事實。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.1
query = '''
3.公司資本及股份 (1)股本來源是否按附表五揭露。
'''
query = '''
公司資本及股份 股本來源 發行價格 核定股本 實收股本
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.2
query = '''
(2)股東結構是否按附表六揭露。
'''
query = '''
股東結構 政府機構 金融機構 其他法人
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.3
query = '''
(3)普通股及特別股之股權分散情形是否按附表七揭露。
'''
query = '''
普通股 特別股 股權分散情形 持股分級 股東人數 持有股數 持股比例
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.4
query = '''
(4)主要股東名單是否按附表八揭露，應列明股權比例達百分之五以上之股東，如不足十名，應揭露至股權比例占前十名之股東名稱、持股數額及比例。
'''
query = '''
主要股東名單 主要股東名稱 持有股數 持股比例
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.5
query = '''
(5)最近二年度每股資料是否按附表九揭露。
'''
query = '''
最近二年度每股資料
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.9.1
query = '''
(9)公司買回本公司股份情形 A.已執行完畢者：是否按附表十(1)揭露。
'''
query = '''
公司買回本公司股份情形 已執行完畢者 買回期次 買回目的
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 2 類: 11.9.2
query = '''
B.尚在執行中者：是否按附表十(2)揭露。
'''
query = '''
尚在執行中者 買回期次 買回目的 買回股份之種類 買回股份之總金額上限
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 4 類: 18.1.1
query = '''
(四)營運概況 1.業務內容 (1)業務範圍。
'''
query = '''
營運概況 業務內容 業務範圍
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 4 類: 18.1.2
query = '''
(2)產業概況。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 4 類: 18.1.3
query = '''
(3)技術及研發概況。
'''
query = '''
技術及研發概況
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')

In [ ]:
# 第 4 類: 18.1.4
query = '''
(4)長、短期業務發展計畫。
'''
sim = bm25_similarity(list_seq, query)
for doc_id, score in get_page_score(sim, top_k):
    print(f'Page Num: {doc_id}, Score: {score:.4f}')